# Optimizing A Simple DNN Training Program Using Nsight Systems
## 6.1 Tensor Core Usage
As part of this lab you have access to a Turing GPU which has Tensor Cores. Tensor Cores are specialized execution units designed specifically for performing the tensor/matrix operations that are the core compute functions used in Deep Learning. The Turing Tensor Cores provide tremendous speed-ups for matrix computations and enable AI programmers to use mixed-precision to achieve higher throughput without sacrificing accuracy.

## 6.2 Analyze the Profile
To verify if the application uses Tensor Cores, we will use a new feature in Nsight Systems: **GPU performance metrics sampling**. Notice that in the Nsight Systems command line used to profile the application after second optimization in section 5.4, we used the `--gpu-metrics-device=all` CLI option. This enables the collection of the new feature. It is intended to measure the utilization of different GPU subsystems. Hardware counters within the GPU are periodically read and used to generate performance metrics.

The data is presented as an intuitive visualization on a timeline which is configurable and easy to navigate. The data is organized in a hierarchical top-down fashion so you can observe the whole profiling session before zooming into individual problem areas. Problems that previously required guessing and testing can now be visually identified at a glance.

Let's analyze the application's Tensor Cores usage by examining the report (secondOptimization.nsys-rep) in the Nsight Systems GUI. Scroll down to the bottom of the timeline until you see the timeline rows for GPU metrics as shown below.

<img src=images/TensorCoreUsage.jpg>

Expand the `SM instructions` timeline row to look at the `Tensor Active/ FP16 Active` which represents the ratio of *cycles the SM tensor pipes or FP16x2 pipes were active issuing tensor instructions* to *the number of cycles in the sample period* as a percentage. The percentage graph is zero here, so the application does not use the Tensor Cores on the Turing GPU at all. This leaves performance on the table when you have access to a powerful Turing GPU.

## 6.3 Optimize the Code to Use Tensor Cores
PyTorch has an [Automatic Mixed Precision (AMP)](https://pytorch.org/docs/stable/amp.html) package which provides a simple way for users to convert existing FP32 training scripts to mixed FP32/FP16 precision. This unlocks faster computation with Tensor Cores on NVIDIA GPUs. Execute the cell below to see the code changes made to use AMP package in PyTorch.

In [ ]:
!diff -U4 --color=always application/main_opt2.py application/main_opt3.py

## 6.4 Profile Again to Verify Optimization
Let's profile again by executing the cell given below to verify if code changes enable Tensor Core usage.

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--gpu-metrics-device=all \
--output /dli/task/nsys/thirdOptimization \
--force-overwrite true \
python3 /dli/task/nsys/application/main_opt3.py

Open the report (thirdOptimization.nsys-rep) in the GUI. Scroll down to view the `Tensor Active / FP16 Active` timeline row.

<img src=images/Optimization3.jpg>

The optimized code indeed uses the Tensor Cores on the Turing GPU.

## 6.5 Compare the Performance Before and After the Optimizations
Now that we have addressed three different performance problems, let's time the application [main_opt3.py](application/main_opt3.py).

In [ ]:
!cd /dli/task/nsys/application && time python3 main_opt3.py

Comparing the time taken to run our baseline code [main_baseline_nvtx.py](application/main_baseline_nvtx.py) from [step 02](02_pytorch_mnist.ipynb) with the code after applying the three optimizations so far [main_opt3.py](application/main_opt3.py), we see that the overall time taken has reduced from ~97s to ~63s.

We could continue the process of optimization by identifying more performance bottlenecks and modifying the application to address them, but in the interest of time we will stop here. You are now ready to move to the [summary](07_summary.ipynb) of this Nsight Systems lab.